

# Proyecto

### Equipo:

- Felipe Jorquera Díaz

- Nombre de usuarios en Codalab: felipejorquera

- Nombre del Equipo en Codalab: felipejorquera

### Link de repositorio de GitHub: `https://github.com/felipejorqueradiaz`




## 1. Introducción

El objetivo de este proyecto consiste en resolver dos problemáticas de negocio asociadas a las aspiraciones de Renacín, el ex-influencer. Ambos problemas se detallan a continuación:
* Problema de clasificación: Renacín desea conocer la potencial evaluación de una película por parte de sus espectadores. Para esto se han construido 5 categorías de puntuación: Negative, Mixed, Mostly Positive, Positive y Very Positive.
* Problema de regresión: Renacín desea conocer las potenciales ganancias de una película, para ver su rentabilidad.


Los datos que proveen es un dataset con 9641 filas y 18 columnas, y luego de un preprocesamiento incial que genera el conjunto de datos que se utilizará en el proyecto finalmente quedan 6451 registros y 13 columnas, dos de las cuales corresponden a las variables objetivo de los dos problemas: la columna numérica "target" representa la recaudación de una película y la columna categórica "label" muestra la clasificación de la misma. De las 11 variables restantes, 2 de ellas son numéricas, una es temporal y otras 8 son categóricas/texto.

El preprocesamiento posterior cumple la función de añadir nuevas variables de interés al problema, lo cual puede mejorar le rendimiento de los modelos. En base a lo anterior es que las variables numéricas son estandarizadas, la variable temporal divida en sus componentes día, mes, año y las variables de texto son codificadas, analizadas y separadas con tal de extraer la mayor información posible.

Para el modelo de clasificación es evaluado por la métrica "F1_macro" que evalúa tanto la "precisión" como el "recall" de las predicciones para cada clase. Además, dado que no todas las clases son igual de frecuentes es que el "F1_macro" pondera de igual forma cada tipo. Para el modelo de regresión se ocupa "R2" que mide el grado de explicabilidad de las variables dependientes sobre la variable independiente.

Para el desarrollo de los modelos, se utilizó un baseline para cada uno con un modelo Dummy y que representa el azar, por lo cual se esperaba que los modelos posteriores pudieran sobrepasar a una asignación aleatoria. Para clasificación se ocuparon Árboles de Decisión, Regresión Logística, Random Forest, KNeighbors y XGBoost, siendo este último el que mayor desempeño mostró en el conjunto de testeo. Para la regresión se ocuparon Regresiones Lineales, Random Forest y XGboost, siendo la regresión lineal la escogida dado que poseía un performance similar al modelo con mejor rendimiento (XGBoost) pero dado que la regresión linal si otorga explicabilidad a las variables y su tiempo de entrenamiento era menor es que se prefirió por este método.

Los resultados de los modelos es bueno dado que superan al baseline desarrollado en este mismo proyecto, pero además superan el baseline impuesto en la competencia, por lo que tuvieron un gran rendimiento. Sin embargo, dada la gran cantidad de variables numéricas es que es posible que los modelos puedan tener un mejor desempeño si se procesa de mejor forma el texto, lo cual por razones de capacidad computacional y tiempo no se pudo realizar.

En métodos generales, el preprocesamiento de variables logró que ambos modelos desarrollados tengan una gran performance a la hora de predecir métricas de interés sobre las películas por lo cual se cumplieron con los objetivos del problema.

---
## 2. Preparación y Análisis Exploratorio de Datos

Por razones de conveniencia de lectura, se ha decidido dejar el Análisis Exploratorio en un documento aparte dada la cantidad de gráficas y código existente.

In [1]:
import pandas as pd
import numpy as np
import umap
import plotly.express as px
import pickle
#pip install pyarrow
#pip install seaborn
#pip install umap-learn

Preprocesamos según lo indicado en el instructivo:

In [2]:
with open('test.pickle', "rb") as fh:
    data_val = pickle.load(fh)
data_val['tipo'] = 'val'

data1 = pd.read_parquet('train_numerical_features.parquet')
data2 = pd.read_parquet('train_text_features.parquet')  
                           
data = data1.merge(data2, on = ['id', 'tagline', 'credits', 'title'])
data['tipo'] = 'traintest'
                           
data = pd.concat([data, data_val], axis=0)
                           
data.drop(['poster_path', 'backdrop_path', 'recommendations'], axis=1, inplace=True)

data = data[~ (
    (data['revenue']==0) |
    (data['release_date'].isna()) |
    (data['runtime'].isna()) |
    ((data['status'] != 'Released'))
                )]

data['release_date'] = pd.to_datetime(data['release_date'], format = '%Y-%m-%d')

data_cat = data.select_dtypes(include=[object])
data[data_cat.columns] = data_cat.fillna('')

data.loc[(data['vote_average'] <= 10) & (data['vote_average'] > 8), 'label'] = 'Very Positive'
data.loc[(data['vote_average'] <= 8) & (data['vote_average'] > 7), 'label'] = 'Positive'
data.loc[(data['vote_average'] <= 7) & (data['vote_average'] > 6), 'label'] = 'Mostly Positive'
data.loc[(data['vote_average'] <= 6) & (data['vote_average'] > 5), 'label'] = 'Mixed'
data.loc[(data['vote_average'] <= 5) , 'label'] = 'Negative'

data.drop(['vote_average', 'id', 'status'], axis=1, inplace=True)
data.rename(columns = {'revenue':'target'},
            inplace=True)

---

## 3. Preprocesamiento, Holdout y Feature Engineering

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')
stop_words = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Añadimos variables temporales:

In [4]:
data['day'] = data['release_date'].dt.day
data['month'] = data['release_date'].dt.month
data['year'] = data['release_date'].dt.year
data['day_name'] = data['release_date'].dt.day_name()

Codificamos solamente el idioma "inglés" dado que es el predominante

In [5]:
data['en_language'] = np.where(data['original_language'] == 'en', 1, 0)

Transformamos la variable budget a logaritmica dada su distribución. Cambiamos los valores indeterminados por cero.

In [6]:
data['log_budget'] = np.log(data['budget'])
data['log_budget'].replace(np.inf, 0, inplace=True)
data['log_budget'].replace(-np.inf, 0, inplace=True)

C:\Users\Felipe\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Para aquellas columnas que muestran más de una "categoría", se agrega la cantidad de valores encontrados. Por ejemplo, cuántos "genres" posee una película.

In [7]:
for col in ['genres', 'credits', 'production_companies', 'keywords']:
    values = [x.split(sep='-') for x in data[col]]
    len_values = [len(x) for x in values]
    data[f'len_{col}'] = len_values

Para la variable "genres", se toman los 10 géneros más frecuentes y se crea una variable nueva para cada uno: 1 si posee el género, 0 si no.
Además, se genera una variable que resume las anteriores e indica si posee un género popular o no. También se crea la variable "Other" en caso de que la pelicula posea un género no popular.

In [8]:
popular_genres = ['Drama', 'Action', 'Comedy', 'Adventure', 'Thriller',
                 'Fantasy', 'Science Fiction', 'Family', 'Crime', 'Romance']
for pop in popular_genres:
    data[f'genre_{pop}'] = pd.DataFrame(np.where(data['genres'].str.contains(pop), 1, 0))
    
data['genre_Other'] = data.loc[:,data.columns[-len(popular_genres):]].sum(1)
data['genre_Popular'] = np.where(data['genre_Other']>0, 1, 0)
data['genre_Other'] = np.where(data['genre_Other']==0, 1, 0)

Creamos una variable binaria que indica si la pelicula pertenece a una de las 50 productoras más famosas.

In [9]:
companies = [x.split(sep='-') for x in data['production_companies']]
values = [i for x in companies for i in x]
val_unique, counts = np.unique(values, return_counts=True)
series_count = pd.Series(counts, index = val_unique).sort_values(ascending = False).head(50)

counter2 = np.zeros(len(data))
for pop in series_count.index:
    counter2 = counter2 + np.where(data['production_companies'].str.contains(pop), 1, 0)
data['Popular_company'] = counter2

Creamos una variable binaria que indica si la pelicula posee a uno de los 150 actores o actrices más famosas.

In [10]:
actores = [x.split(sep='-') for x in data['credits']]
values = [i for x in actores for i in x]
val_unique, counts = np.unique(values, return_counts=True)
series_count = pd.Series(counts, index = val_unique).sort_values(ascending = False).head(150)

counter2 = np.zeros(len(data))
for pop in series_count.index:
    counter2 = counter2 + np.where(data['credits'].str.contains(pop), 1, 0)
data['Popular_actor'] = counter2

Ahora creamos el Tokenizer pera procesar texto

In [11]:
#Seteamos el idioma en español
stop_words = stopwords.words('spanish')

class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

Creamos la Pipeline con los preprocesamientos para columnas numéricas y texto

In [12]:
col_num = ['budget', 'runtime', 'log_budget']

preprocessing = ColumnTransformer(
    transformers = [
        ('StandardScaler', StandardScaler(), col_num),
        ('OneHotEncoder', OneHotEncoder(), ['day_name']),
        ('BagOfWords1', CountVectorizer(tokenizer= StemmerTokenizer(),
                                               ngram_range=(1,2) 
                                              ), 'title'),
        ('BagOfWords2', CountVectorizer(tokenizer= StemmerTokenizer(),
                                               ngram_range=(1,2) 
                                              ), 'overview'),
        ('BagOfWords3', CountVectorizer(tokenizer= StemmerTokenizer(),
                                               ngram_range=(1,2) 
                                              ), 'keywords')
    ])

Separamos los datos nuevamente

In [13]:
data_val = data[data['tipo'] == 'val'].drop('tipo', axis=1)
data = data[data['tipo'] != 'val'].drop('tipo', axis=1)

Hacemos LabelEncoder para que XGBoost pueda soportarlo

In [14]:
le = LabelEncoder()
le.fit(data['label'])
data['label'] = le.transform(data['label'])

Razonamiento:

Con las variables numéricas se aplicó una estandarización para lograr crear una distribución más suave y que evitara outliers. Además, para el texto se crearon variables como la cantidad de géneros, de actores, de productoras, etc, con tal de extraer información de las columnas que mostraban muchas categorías al mismo tiempo. Para el texto nato, se decidió aplicar un stemmizador para vectorizar y así extraer información de dichas palabras.

---

## 4. Clasificación

### 4.1 Dummy y Baseline

In [15]:
from sklearn.dummy import DummyClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report

#pip install xgboost

Separamos los datos en train y test:

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['target', 'credits', 'genres', 'tagline',
       'original_language', 'production_companies', 'release_date', 'label'], axis=1),
    data['label'],
    shuffle = True,
    test_size = 0.3,
    random_state = 0
)

Creamos el baseline y un modelo simple, en este caso un DecisionTree.

In [18]:
pipe_clf_baseline = Pipeline(steps = [
    ('prepro', preprocessing),
    ('modelo', DummyClassifier(strategy = 'stratified'))
])

pipe_clf_simplemodel = Pipeline(steps = [
    ('prepro', preprocessing),
    ('modelo', DecisionTreeClassifier(random_state=0))
])

Ejecutamos y vemos las métricas

In [19]:
pipelines = {'Baseline': pipe_clf_baseline,
            'DecisionTree': pipe_clf_simplemodel}

for name, model in pipelines.items():
    model.fit(X_train, y_train)
    y_pred = le.inverse_transform(model.predict(X_test))
    
    scores = classification_report(le.inverse_transform(y_test), y_pred)
    print(f'Para el modelo {name} el performance es el siguiente:\n',scores,'\n\n')

Para el modelo Baseline el performance es el siguiente:
                  precision    recall  f1-score   support

          Mixed       0.22      0.21      0.21       417
Mostly Positive       0.46      0.45      0.45       908
       Negative       0.02      0.02      0.02        47
       Positive       0.27      0.28      0.27       514
  Very Positive       0.06      0.06      0.06        50

       accuracy                           0.33      1936
      macro avg       0.20      0.20      0.20      1936
   weighted avg       0.33      0.33      0.33      1936
 


Para el modelo DecisionTree el performance es el siguiente:
                  precision    recall  f1-score   support

          Mixed       0.31      0.27      0.29       417
Mostly Positive       0.49      0.56      0.52       908
       Negative       0.03      0.02      0.03        47
       Positive       0.41      0.40      0.41       514
  Very Positive       0.08      0.04      0.05        50

       accuracy    

El DecisionTree es indudablemente mejor que el modelo Dummy ya que permite identificar con mayor exactitud las clases "Positive", "Mixed" y "Mostly Positive". Sin embargo, la mejora de la métrica "f1-macro" no es enorme por lo que existe un gran espacio de mejora para probar modelos más complejos y probar combinaciones de hiper-parámetros.

---

### 4.2 Búsqueda del mejor modelo de Clasificación


Creamos la pipeline para el GridSearch:

In [28]:
pipe_clf_grid = Pipeline(steps = [
    ('prepro', preprocessing),
    ('modelo', DecisionTreeClassifier())
])

Generamos los parámetros a probar:

In [29]:
params = [
    {
        'modelo': [LogisticRegression(random_state = 0)],
        'modelo__penalty': ['l1', 'l2'],
        'modelo__solver': ['liblinear'],
        'prepro__BagOfWords2__ngram_range': [(1,1), (1,2)],
        'prepro__BagOfWords3__ngram_range': [(1,1), (1,2)]
    },{
        'modelo': [RandomForestClassifier(random_state=0)],
        'modelo__max_depth': [2,4],
        'prepro__BagOfWords2__ngram_range': [(1,1), (1,2)]
    },{
        'modelo': [xgb.XGBClassifier(random_state=0)],
        'prepro__BagOfWords2__ngram_range': [(1,1), (1,2)],
        'prepro__BagOfWords3__ngram_range': [(1,1), (1,2)]
    },{
        'modelo': [KNeighborsClassifier()],
        'modelo__n_neighbors': [2,3,4],
        'prepro__BagOfWords2__ngram_range': [(1,1), (1,2)],
        'prepro__BagOfWords3__ngram_range': [(1,1), (1,2)]
    }]

Entrenamos

In [30]:
grid_model_clf = GridSearchCV(pipe_clf_grid,
                          param_grid=params,
                          verbose=10,
                          scoring = 'f1_macro',
                          cv=3)

grid_model_clf.fit(X_train, y_train)

Fitting 3 folds for each of 28 candidates, totalling 84 fits
[CV 1/3; 1/28] START modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 1)
[CV 1/3; 1/28] END modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 1);, score=0.250 total time=   9.8s
[CV 2/3; 1/28] START modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 1)
[CV 2/3; 1/28] END modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 1);, score=0.246 total time=   9.3s
[CV 3/3; 1/28] START modelo=LogisticRegression(random_state=0), modelo__penalty=l1, modelo__solver=liblinear, pre

[CV 3/3; 7/28] END modelo=LogisticRegression(random_state=0), modelo__penalty=l2, modelo__solver=liblinear, prepro__BagOfWords2__ngram_range=(1, 2), prepro__BagOfWords3__ngram_range=(1, 1);, score=0.237 total time=  11.8s
[CV 1/3; 8/28] START modelo=LogisticRegression(random_state=0), modelo__penalty=l2, modelo__solver=liblinear, prepro__BagOfWords2__ngram_range=(1, 2), prepro__BagOfWords3__ngram_range=(1, 2)
[CV 1/3; 8/28] END modelo=LogisticRegression(random_state=0), modelo__penalty=l2, modelo__solver=liblinear, prepro__BagOfWords2__ngram_range=(1, 2), prepro__BagOfWords3__ngram_range=(1, 2);, score=0.253 total time=  11.2s
[CV 2/3; 8/28] START modelo=LogisticRegression(random_state=0), modelo__penalty=l2, modelo__solver=liblinear, prepro__BagOfWords2__ngram_range=(1, 2), prepro__BagOfWords3__ngram_range=(1, 2)
[CV 2/3; 8/28] END modelo=LogisticRegression(random_state=0), modelo__penalty=l2, modelo__solver=liblinear, prepro__BagOfWords2__ngram_range=(1, 2), prepro__BagOfWords3__ngra

[CV 3/3; 13/28] END modelo=XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=0,
              reg_alpha=None, reg_lambda=None, ...), prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 1);, score=0.256 total time=  19.1s
[CV 1/3; 14/28] START modelo=XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_byleve

[CV 2/3; 15/28] END modelo=XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=0,
              reg_alpha=None, reg_lambda=None, ...), prepro__BagOfWords2__ngram_range=(1, 2), prepro__BagOfWords3__ngram_range=(1, 1);, score=0.257 total time=  33.5s
[CV 3/3; 15/28] START modelo=XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_byleve

[CV 2/3; 17/28] END modelo=KNeighborsClassifier(), modelo__n_neighbors=2, prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 1);, score=0.187 total time=  10.4s
[CV 3/3; 17/28] START modelo=KNeighborsClassifier(), modelo__n_neighbors=2, prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 1)
[CV 3/3; 17/28] END modelo=KNeighborsClassifier(), modelo__n_neighbors=2, prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 1);, score=0.189 total time=   9.6s
[CV 1/3; 18/28] START modelo=KNeighborsClassifier(), modelo__n_neighbors=2, prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 2)
[CV 1/3; 18/28] END modelo=KNeighborsClassifier(), modelo__n_neighbors=2, prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 2);, score=0.198 total time=  10.6s
[CV 2/3; 18/28] START modelo=KNeighborsClassifier(), modelo__n_neighbors=2, prepro__BagOfWords2__ngram_range=(1, 1), prepr

[CV 2/3; 25/28] END modelo=KNeighborsClassifier(), modelo__n_neighbors=4, prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 1);, score=0.194 total time=  10.0s
[CV 3/3; 25/28] START modelo=KNeighborsClassifier(), modelo__n_neighbors=4, prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 1)
[CV 3/3; 25/28] END modelo=KNeighborsClassifier(), modelo__n_neighbors=4, prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 1);, score=0.199 total time=  10.0s
[CV 1/3; 26/28] START modelo=KNeighborsClassifier(), modelo__n_neighbors=4, prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 2)
[CV 1/3; 26/28] END modelo=KNeighborsClassifier(), modelo__n_neighbors=4, prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 2);, score=0.200 total time=  10.6s
[CV 2/3; 26/28] START modelo=KNeighborsClassifier(), modelo__n_neighbors=4, prepro__BagOfWords2__ngram_range=(1, 1), prepr

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('prepro',
                                        ColumnTransformer(transformers=[('StandardScaler',
                                                                         StandardScaler(),
                                                                         ['budget',
                                                                          'runtime',
                                                                          'log_budget']),
                                                                        ('OneHotEncoder',
                                                                         OneHotEncoder(),
                                                                         ['day_name']),
                                                                        ('BagOfWords1',
                                                                         CountVectorizer(ngram_range=(1,
                                                                                                      2),
                                                                                         tokenizer=<__main__.StemmerTokenizer object at 0x000001AAAB6D28E0>),
                                                                         'title'),
                                                                        ('BagOfWords2',
                                                                         CountVect...
                                                   predictor=None,
                                                   random_state=0,
                                                   reg_alpha=None,
                                                   reg_lambda=None, ...)],
                          'prepro__BagOfWords2__ngram_range': [(1, 1), (1, 2)],
                          'prepro__BagOfWords3__ngram_range': [(1, 1), (1, 2)]},
                         {'modelo': [KNeighborsClassifier()],
                          'modelo__n_neighbors': [2, 3, 4],
                          'prepro__BagOfWords2__ngram_range': [(1, 1), (1, 2)],
                          'prepro__BagOfWords3__ngram_range': [(1, 1),
                                                               (1, 2)]}],
             scoring='f1_macro', verbose=10)

Visualizamos el mejor modelo:

In [31]:
y_pred = le.inverse_transform(grid_model_clf.predict(X_test))
print(classification_report(le.inverse_transform(y_test), y_pred))

                 precision    recall  f1-score   support

          Mixed       0.37      0.16      0.22       417
Mostly Positive       0.50      0.73      0.59       908
       Negative       0.00      0.00      0.00        47
       Positive       0.50      0.42      0.46       514
  Very Positive       0.00      0.00      0.00        50

       accuracy                           0.49      1936
      macro avg       0.27      0.26      0.25      1936
   weighted avg       0.45      0.49      0.45      1936



El TOP 5 de modelos encontrados:

In [32]:
pd.DataFrame(grid_model_clf.cv_results_).sort_values(by='rank_test_score').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_modelo,param_modelo__penalty,param_modelo__solver,param_prepro__BagOfWords2__ngram_range,param_prepro__BagOfWords3__ngram_range,param_modelo__max_depth,param_modelo__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
15,31.077326,0.134227,3.785813,0.304545,"XGBClassifier(base_score=None, booster=None, c...",NaN,NaN,"(1, 2)","(1, 2)",NaN,NaN,"{'modelo': XGBClassifier(base_score=None, boos...",0.268943,0.265005,0.266767,0.266905,0.001611,1
14,28.727118,0.444869,4.202138,0.096296,"XGBClassifier(base_score=None, booster=None, c...",NaN,NaN,"(1, 2)","(1, 1)",NaN,NaN,"{'modelo': XGBClassifier(base_score=None, boos...",0.265651,0.257181,0.269544,0.264125,0.005161,2
13,18.139403,0.752783,3.468434,0.109548,"XGBClassifier(base_score=None, booster=None, c...",NaN,NaN,"(1, 1)","(1, 2)",NaN,NaN,"{'modelo': XGBClassifier(base_score=None, boos...",0.256727,0.258870,0.266415,0.260671,0.004155,3
12,15.595951,0.181515,3.478224,0.191650,"XGBClassifier(base_score=None, booster=None, c...",NaN,NaN,"(1, 1)","(1, 1)",NaN,NaN,"{'modelo': XGBClassifier(base_score=None, boos...",0.265333,0.249660,0.255944,0.256979,0.006440,4
4,7.164072,0.285320,3.306566,0.080994,LogisticRegression(random_state=0),l2,liblinear,"(1, 1)","(1, 1)",NaN,NaN,"{'modelo': LogisticRegression(random_state=0),...",0.259821,0.251751,0.244822,0.252131,0.006129,5


In [33]:
yc_val = grid_model_clf.predict(data_val.drop(['target', 'credits', 'genres', 'tagline',
       'original_language', 'production_companies', 'release_date', 'label'], axis=1))

yc_val = le.inverse_transform(yc_val)

En el GridSearch se probaron distintas configuraciones, principalmente en la forma de procesar el texto dada la cantidad de variables relacionadas, por lo cual el proceso fue muy largo y costoso computacionlmente. Los modelos probados fueron: RandomForest, XGBoost, KNeighbors y Regresión Logistica. 

Luego de probar cada modelo, se llega a la conclusión de que el mejor modelo es el XGBoost dado que 4 configuraciones distintas poseen los 5 mejores rendimientos en el set de testeo.

---

## 5. Regresión

### 5.1 Dummy y Baseline

In [34]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
from sklearn.metrics import r2_score

Separamos los datos:

In [36]:
Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    data.drop(['target', 'tagline', 'credits', 'genres',
       'original_language', 'production_companies', 'release_date', 'label'], axis=1),
    data['target'],
    shuffle = True,
    test_size = 0.3,
    random_state = 0
)

Generamos los pipelines para el baseline y el modelo simple (Regresión Lineal)

In [37]:
pipe_reg_baseline = Pipeline(steps = [
    ('prepro', preprocessing),
    ('model', DummyRegressor())
])

pipe_reg_simplemodel = Pipeline(steps = [
    ('prepro', preprocessing),
    ('model', LinearRegression())
])

Entrenamos:

In [38]:
pipelines = {'Baseline':pipe_reg_baseline,
             'OLS': pipe_reg_simplemodel}

for name, model in pipelines.items():
    model.fit(Xr_train, yr_train)
    yr_pred = model.predict(Xr_test)
    
    scores = r2_score(yr_test, yr_pred)
    print(f'Para el modelo {name} el performance es el siguiente:\n',np.round(scores,3),'\n\n')

Para el modelo Baseline el performance es el siguiente:
 -0.0 


Para el modelo OLS el performance es el siguiente:
 0.525 




Claramente el modelo Dummy tiene un mal performance al llegar a un R2 negativo. Afortunadamente, la regresión lineal ha sido altamente competente y entrega un R2 muy alto, mayor al 0.5 lo cual es un gran performance para un modelo tan simple y que no requiere una parametrización.

---

### 5.2 Búsqueda del mejor modelo de Regresión


Creamos la pipeline base para el GridSearch:

In [39]:
pipe_reg_grid = Pipeline(steps = [
    ('prepro', preprocessing),
    ('modelo', LinearRegression())
])

Cambiamos parámetros de interés:

In [40]:
params = [
    {
        'modelo': [LinearRegression()],
        'prepro__BagOfWords2__ngram_range': [(1,1), (1,2), (1,3)],
        'prepro__BagOfWords3__ngram_range': [(1,1), (1,2), (1,3)]
    },{
        'modelo': [RandomForestRegressor(random_state=0)],
        'modelo__max_depth': [2,4],
        'prepro__BagOfWords2__ngram_range': [(1,1), (1,2)],
        'prepro__BagOfWords3__ngram_range': [(1,1), (1,2)]
    },{
        'modelo': [xgb.XGBRegressor(random_state=0)],
        'prepro__BagOfWords2__ngram_range': [(1,1), (1,2), (1,3)],
        'prepro__BagOfWords3__ngram_range': [(1,1), (1,2), (1,3)]
    }]

Entrenamos:

In [41]:
grid_model_reg = GridSearchCV(pipe_reg_grid,
                          param_grid=params,
                          verbose=10,
                          scoring = 'r2',
                          cv=3)

grid_model_reg.fit(Xr_train, yr_train)

Fitting 3 folds for each of 26 candidates, totalling 78 fits
[CV 1/3; 1/26] START modelo=LinearRegression(), prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 1)
[CV 1/3; 1/26] END modelo=LinearRegression(), prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 1);, score=0.466 total time=  10.1s
[CV 2/3; 1/26] START modelo=LinearRegression(), prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 1)
[CV 2/3; 1/26] END modelo=LinearRegression(), prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 1);, score=0.508 total time=   9.5s
[CV 3/3; 1/26] START modelo=LinearRegression(), prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 1)
[CV 3/3; 1/26] END modelo=LinearRegression(), prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 1);, score=0.519 total time=   9.1s
[CV 1/3; 2/26] START modelo=LinearRegression(), prepro__BagOfWords2__ngr

[CV 2/3; 10/26] END modelo=RandomForestRegressor(random_state=0), modelo__max_depth=2, prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 1);, score=0.540 total time=  16.0s
[CV 3/3; 10/26] START modelo=RandomForestRegressor(random_state=0), modelo__max_depth=2, prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 1)
[CV 3/3; 10/26] END modelo=RandomForestRegressor(random_state=0), modelo__max_depth=2, prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 1);, score=0.588 total time=  14.6s
[CV 1/3; 11/26] START modelo=RandomForestRegressor(random_state=0), modelo__max_depth=2, prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 2)
[CV 1/3; 11/26] END modelo=RandomForestRegressor(random_state=0), modelo__max_depth=2, prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 2);, score=0.493 total time=  13.4s
[CV 2/3; 11/26] START modelo=RandomForestRegressor(random

[CV 1/3; 18/26] END modelo=XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=None,
             gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=0,
             reg_alpha=None, reg_lambda=None, ...), prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 1);, score=0.512 total time=  16.4s
[CV 2/3; 18/26] START modelo=XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsa

[CV 3/3; 19/26] END modelo=XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=None,
             gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=0,
             reg_alpha=None, reg_lambda=None, ...), prepro__BagOfWords2__ngram_range=(1, 1), prepro__BagOfWords3__ngram_range=(1, 2);, score=0.608 total time=  13.1s
[CV 1/3; 20/26] START modelo=XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsa

[CV 2/3; 21/26] END modelo=XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=None,
             gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=0,
             reg_alpha=None, reg_lambda=None, ...), prepro__BagOfWords2__ngram_range=(1, 2), prepro__BagOfWords3__ngram_range=(1, 1);, score=0.549 total time=  16.9s
[CV 3/3; 21/26] START modelo=XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsa

[CV 1/3; 23/26] END modelo=XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=None,
             gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=0,
             reg_alpha=None, reg_lambda=None, ...), prepro__BagOfWords2__ngram_range=(1, 2), prepro__BagOfWords3__ngram_range=(1, 3);, score=0.528 total time=  18.3s
[CV 2/3; 23/26] START modelo=XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsa

[CV 3/3; 24/26] END modelo=XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=None,
             gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=0,
             reg_alpha=None, reg_lambda=None, ...), prepro__BagOfWords2__ngram_range=(1, 3), prepro__BagOfWords3__ngram_range=(1, 1);, score=0.602 total time=  24.9s
[CV 1/3; 25/26] START modelo=XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsa

[CV 2/3; 26/26] END modelo=XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=None,
             gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=0,
             reg_alpha=None, reg_lambda=None, ...), prepro__BagOfWords2__ngram_range=(1, 3), prepro__BagOfWords3__ngram_range=(1, 3);, score=0.541 total time=  24.6s
[CV 3/3; 26/26] START modelo=XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsa

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('prepro',
                                        ColumnTransformer(transformers=[('StandardScaler',
                                                                         StandardScaler(),
                                                                         ['budget',
                                                                          'runtime',
                                                                          'log_budget']),
                                                                        ('OneHotEncoder',
                                                                         OneHotEncoder(),
                                                                         ['day_name']),
                                                                        ('BagOfWords1',
                                                                         CountVectorizer(ngram_range=(1,
                                                                                                      2),
                                                                                         tokenizer=<__main__.StemmerTokenizer object at 0x000001AAAB6D28E0>),
                                                                         'title'),
                                                                        ('BagOfWords2',
                                                                         CountVect...
                                                  max_delta_step=None,
                                                  max_depth=None,
                                                  max_leaves=None,
                                                  min_child_weight=None,
                                                  missing=nan,
                                                  monotone_constraints=None,
                                                  n_estimators=100, n_jobs=None,
                                                  num_parallel_tree=None,
                                                  predictor=None,
                                                  random_state=0,
                                                  reg_alpha=None,
                                                  reg_lambda=None, ...)],
                          'prepro__BagOfWords2__ngram_range': [(1, 1), (1, 2),
                                                               (1, 3)],
                          'prepro__BagOfWords3__ngram_range': [(1, 1), (1, 2),
                                                               (1, 3)]}],
             scoring='r2', verbose=10)

Vemos el score del mejor modelo:

In [43]:
pd.DataFrame(grid_model_reg.cv_results_).sort_values('rank_test_score').head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_modelo,param_prepro__BagOfWords2__ngram_range,param_prepro__BagOfWords3__ngram_range,param_modelo__max_depth,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
14,14.039904,0.257480,3.286812,0.205433,"RandomForestRegressor(max_depth=4, random_stat...","(1, 1)","(1, 2)",4,"{'modelo': RandomForestRegressor(max_depth=4, ...",0.517888,0.576925,0.609210,0.568008,0.037811,1
8,11.084009,0.605124,3.927984,0.564064,LinearRegression(),"(1, 3)","(1, 3)",NaN,"{'modelo': LinearRegression(), 'prepro__BagOfW...",0.529381,0.574226,0.598862,0.567490,0.028763,2
13,12.267656,0.118202,3.816814,0.072554,"RandomForestRegressor(max_depth=4, random_stat...","(1, 1)","(1, 1)",4,"{'modelo': RandomForestRegressor(max_depth=4, ...",0.516457,0.576731,0.608192,0.567127,0.038062,3
7,9.721029,0.410769,3.844726,0.429512,LinearRegression(),"(1, 3)","(1, 2)",NaN,"{'modelo': LinearRegression(), 'prepro__BagOfW...",0.527662,0.571952,0.596060,0.565225,0.028326,4
17,10.707795,1.575881,3.379925,0.145755,"XGBRegressor(base_score=None, booster=None, ca...","(1, 1)","(1, 1)",NaN,"{'modelo': XGBRegressor(base_score=None, boost...",0.512233,0.573618,0.604603,0.563485,0.038385,5


Tomamos el modelo que tiene un mejor performance:

In [62]:
pipe_reg_grid_best = Pipeline(steps = [
    ('prepro', preprocessing),
    ('modelo', LinearRegression())
])

pipe_reg_grid_best.set_params(prepro__BagOfWords2__ngram_range=(1,3))
pipe_reg_grid_best.set_params(prepro__BagOfWords3__ngram_range=(1,3))

pipe_reg_grid_best.fit(Xr_train, yr_train)
yr_pred = pipe_reg_grid_best.predict(Xr_test)

print(r2_score(yr_test, yr_pred))

0.5371998630857908


In [50]:
yr_val = pipe_reg_grid_best.predict(data_val.drop(['target', 'credits', 'genres',
                                       'original_language', 'production_companies',
                                      'release_date', 'label'], axis=1))

La regresión lineal trae unos grandes resultados en comparación a modelos mucho más complejos y costosos, por lo que se opta por ese modelo y no por el RandomForest que tiene una performance milimétricamente mayor. Los mejores modelos oscilan en un R2 cercano a 0.59 en el set de testo por lo cual se mejora lo obtenido en la sección anterior, sin embargo el coste computacional de encontrar los parámetros fue muy alto.

---

## 6. Conclusiones

El problema de negocio era generar modelos de predicción para variables categóricas (clasificación) y numéricas (regresión). Con los rendimientos obtenidos en las secciones anteriores se puede decir que **si se cumplieron con los objetivos** planteados. En todos los casos, el modelo genera un output que es mejor que la asignación aleatoria y que además es mejor que el baseline de la competencia, por lo que los resultados son aceptables.

La generación de estos modelos comenzó con el análisis exploratorio, en donde se comprendieron las diversas relaciones que existían entre las variables, principalmente aquellos atributos que denotaban texto. De esta forma se pudo simplificar cierta información y generar nuevas variables que ayudaran a resolver el problema. Uno de los puntos más complejos fue la codificación de texto que por naturaleza es más costoso computacionalmente y es díficil obtener información valiosa si no se aplican los métodos correctos.

Hablando ya de los modelos, sobre la clasficación se obtuvo un baseline cercano a los 0.25 con un DecisionTree, algo levemente mayor a la asignación aleatoria, por lo que se aplican técnicas de elección de hiperparámetros para poder mejorar el performance. En esta grilla, se ocuparon los modelos Regresión Logística, Random Forest, KNeighbors y XGBoost con cambios en la complejidad de los algoritmos como perturbaciones en la profundidad del análisis del texto. Esta búsqueda demora cerca de 40 minutos en ejecutarse y el rendimiento óptimo se lo lleva el XGBoost, pero que no es muy superior al baseline y dificilmente llega a los 0.3.

Para el modelo de regresión el razonamiento fue similar. Se ocupó como baseline una regresión lineal que tenía como R2 base 0.53, algo bastante decente para ser el modelo tan simple. Nuevamente se ocupó GridSearch para buscar una mejor combinación entre los modelos Regresión Lineal, RandomForest y XGBoost, siendo el RandomForest el de mejor performance. Sin embargo, dada la complejidad, la pérdida parcial de interpretabilidad y tiempo en entrenar se opta por el segundo mejor modelo: Regresión Lineal. Notar que la diferencia entre ambos R2 es de 0.00051. La mejora producida por el GridSearch es muy menor a lo esperado y tiene un peak de 0.568, bastate cercano a los 0.53 del baseline.

En base a todo lo anterior, es que se puede señalar que los resultados son buenos, pero no óptimos, dado que se puedieron crear muchísimas más variables en la etapa de preprocesamiento si se contara con el conocimiento en procesamiento de lenguaje. La gran cantidad de columnas de texto hizo dificil la creación de nuevos atributos, pero es posible que si se extrayera mayor información el performance de los modelos sea mejor.

También, el modelo de clasificación, el modelo usualmente confundía a las clases positivas, dada su cercanía en puntaje. Para un mejor modelamiento se cree que reagrupar las variables de puntuación es lo más conveniente (por ej: Negativa, Mix, Positiva) dado que se pueden diferenciar mejor los comportamientos de los usuarios si es que no se segrega con las puntuaciones intermedias.

A métodos generales, el proyecto ha sido una gran oportunidad de poner en práctica los distintos conocimientos en ciencia de datos que se han aprendido en cursos anteriores como en este curso práctico. En retrospectiva, uno de los elementos que personalmente ha sido más complejo es el procesamiento de texto, el cual es una rama completa de estudio en la ciencia de datos y que se espera poder profundizar en cursos posteriores, sin embargo, el curso en sí es bastante completo en los tópicos que se abarcan y lograr generan un aprendizaje rápido y continuo en los temás más esenciales del Data Science.

---

<br>

### Anexo: Generación de Archivo Submit de la Competencia

Para subir los resultados obtenidos a la pagina de CodaLab utilice la función `generateFiles` entregada mas abajo. Esto es debido a que usted deberá generar archivos que respeten extrictamente el formato de CodaLab, de lo contario los resultados no se veran reflejados en la pagina de la competencia.

Para los resultados obtenidos en su modelo de clasificación y regresión, estos serán guardados en un archivo zip que contenga los archivos `predicctions_clf.txt` para la clasificación y `predicctions_rgr.clf` para la regresión. Los resultados, como se comento antes, deberan ser obtenidos en base al dataset `test.pickle` y en cada una de las lineas deberan presentar las predicciones realizadas.

Ejemplos de archivos:

- [ ] `predicctions_clf.txt`

        Mostly Positive
        Mostly Positive
        Negative
        Positive
        Negative
        Positive
        ...

- [ ] `predicctions_rgr.txt`

        16103.58
        16103.58
        16041.89
        9328.62
        107976.03
        194374.08
        ...



In [59]:
from zipfile import ZipFile
import os

def generateFiles(data, clf_pipe, rgr_pipe):
    """Genera los archivos a subir en CodaLab

    Input
    predict_data: Dataframe con los datos de entrada a predecir
    clf_pipe: pipeline del clf
    rgr_pipe: pipeline del rgr

    Ouput
    archivo de txt
    """
    y_pred_clf = le.inverse_transform(clf_pipe.predict(data))
    y_pred_rgr = rgr_pipe.predict(data)
    
    with open('./predictions_clf.txt', 'w') as f:
        for item in y_pred_clf:
            f.write("%s\n" % item)

    with open('./predictions_rgr.txt', 'w') as f:
        for item in y_pred_rgr:
            f.write("%s\n" % item)

    with ZipFile('predictions.zip', 'w') as zipObj2:
       zipObj2.write('predictions_rgr.txt')
       zipObj2.write('predictions_clf.txt')

    os.remove("predictions_rgr.txt")
    os.remove("predictions_clf.txt")

In [60]:
data_val2 = data_val.drop(['target', 'credits', 'genres',
               'original_language', 'production_companies',
               'release_date', 'label'], axis=1)

In [61]:
# Ejecutar función para generar el archivo de predicciones.
# perdict_data debe tener cargada los datos del text.pickle
# mientras que clf_pipe y rgr_pipe, son los pipeline de 
# clasificación y regresión respectivamente.
generateFiles(data_val2, grid_model_clf, pipe_reg_grid_best)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>